<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please do not use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


### 🏦 Checking Your Polygon API Plan

The following code cell determines your Polygon API plan and prints whether you have a **paid** 💳 or **realtime** ⏱️ subscription.

---

### ⚙️ Setting Up MCP Servers for the Trader

Below is the configuration for the MCP servers that your trading agent will use. These servers handle account management, push notifications, and market data integration. 🚀📈

In [ ]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]}) # Caching the data so we don't exceed our rate limits

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### 🕵️‍♂️ Meet the Researcher Agent!

Our next agent is the **Researcher** 🧑‍💻, designed to perform in-depth market research and uncover valuable trading insights. 

---

#### 🛠️ Tools for the Researcher

- **Brave Search API** 🔎: Empowers the agent to search the web for the latest financial news and trends.
- **Fetch Capability** 🌐: Allows the agent to retrieve and analyze web pages for up-to-date information.

With these tools, the Researcher agent can:
- Explore financial news sources 📰
- Identify emerging trading opportunities 📈
- Provide comprehensive market research reports 📊

Let's set up the Researcher agent with these powerful capabilities! 🚀

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

# Define the parameters for launching the MCP servers that the researcher agent will use.
# This is a list of dictionaries, where each dictionary specifies how to start a particular MCP server process.

researcher_mcp_server_params = [
    # The first MCP server is started using the "uvx" command with the argument "mcp-server-fetch".
    # "uvx" is likely a command runner or environment, and "mcp-server-fetch" is the server to be launched.
    {"command": "uvx", "args": ["mcp-server-fetch"]},

    # The second MCP server is started using the "npx" command, which is a Node.js package runner.
    # The "-y" flag automatically confirms any prompts, and "@modelcontextprotocol/server-brave-search" is the package to run.
    # The "env" key sets environment variables for this process, specifically passing in the Brave API key via brave_env.
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

🛠️ **Creating MCPServerStdio Instances for the Researcher and Trader Agents** 🛠️

In the next code cell, we are:

1. **Launching MCP Servers for the Researcher Agent** 🧑‍💻🔎  
- We use the `researcher_mcp_server_params` list to start each MCP server process needed by the Researcher agent.
- Each server is wrapped in an `MCPServerStdio` instance, which manages communication via standard input/output.
- The `client_session_timeout_seconds=30` parameter ensures that each session will timeout after 30 seconds of inactivity.

2. **Launching MCP Servers for the Trader Agent** 💹🤖  
- Similarly, we use `trader_mcp_server_params` to start the MCP servers for the Trader agent.
- Each is also wrapped in an `MCPServerStdio` instance with the same session timeout.

3. **Combining All MCP Servers** 🔗  
- We concatenate the lists of trader and researcher MCP servers into a single `mcp_servers` list.
- This unified list can be used to provide both agents with access to all their required tools and capabilities.

---

🚦 **Why is this important?**  
By structuring the MCP servers this way, each agent can independently access the tools it needs, while also making it easy to manage and scale the system as new tools or agents are added!



In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

## 🧑‍💻🔍 Researcher Agent: Your AI-Powered Market Analyst!

In the next cell, we define an **asynchronous function** to create a "Researcher" agent, powered by GPT-4, that can search the web for financial news, uncover trading opportunities, and summarize its findings for you.  

### 🚀 What does this cell do?
- **Creates a specialized agent**: The function `get_researcher` sets up an agent with clear instructions to act as a financial researcher.
- **Provides context**: It includes the current date and time, so the agent's research is always up-to-date.
- **Connects to MCP servers**: The agent is given access to the right set of tools and data sources via the `mcp_servers` parameter.

### 🌟 Why is this helpful?
- **Automates market research**: Instead of manually searching for news or opportunities, you can delegate this task to an AI agent.
- **Ensures comprehensive results**: The agent is instructed to perform multiple searches and synthesize a broad overview, not just a single headline.
- **Flexible usage**: Whether you have a specific question or just want the latest opportunities, the agent adapts to your needs.
- **Seamless integration**: By wrapping the agent as a tool, it can be easily used by other agents (like a Trader) or in automated workflows.

### ✨ TL;DR
This cell sets up your own AI-powered financial researcher, making it easy to get timely, relevant, and actionable market insights—no manual Googling required! 📈🤖

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

## 🤖🔧 Why are we converting the Researcher agent into a tool in this cell?

In this cell, we are preparing to wrap the Researcher agent as a tool so that it can be easily used by other agents or workflows. By converting the agent into a tool, we make it accessible through a standardized interface, allowing it to be called programmatically as part of a larger agentic system.

### ✨ **Analyzing the next cell:**  
The next cell (`get_researcher_tool`) actually performs this conversion: it asynchronously creates the Researcher agent and then exposes it as a tool using the `.as_tool()` method. This is important because, in subsequent code, other agents (like a Trader agent) or orchestrators can invoke the Researcher as a tool to perform research tasks on demand, rather than interacting with it directly as a standalone agent.

🧩 In summary, converting the Researcher agent into a tool in this cell sets up a modular, reusable component that can be integrated into more complex agent workflows, as demonstrated in the next cell.



In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

## 🧪 Directly Testing the Researcher Agent

Before wrapping the Researcher agent as a tool, let's first call it directly to ensure it works as expected.

We'll ask the following research question:

> **What's the latest news on Amazon?**

In [8]:
# Define the research question that will be sent to the Researcher agent.
research_question = "What's the latest news on Amazon?"

# Iterate over each MCP server instance in the list of researcher MCP servers.
for server in researcher_mcp_servers:
    # Asynchronously establish a connection to the MCP server.
    await server.connect()

# Asynchronously create or retrieve a Researcher agent instance, passing in the connected MCP servers.
researcher = await get_researcher(researcher_mcp_servers)

# Use a tracing context (for debugging or logging) labeled "Researcher" to monitor the following block.
with trace("Researcher"):
    # Asynchronously run the Researcher agent with the research question, allowing up to 30 conversational turns.
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



The latest news on Amazon focuses heavily on their upcoming Prime Day 2025 event. Here are the key points:

- Prime Day 2025 is scheduled for July 8-11, extending to four days instead of the usual two.
- Prime Day will feature exclusive daily deals from top brands like Samsung, Kiehl's, and Levi's.
- This event will be available in 26 countries, including Ireland and Colombia for the first time.
- Prime members can enjoy deep discounts across more than 35 categories.
- Amazon is promoting ways to support U.S. small businesses during Prime Day.
- Early deals include promotions such as 3 free months of Kindle Unlimited for book lovers.
- The deals will run through to Friday, July 11, 11:59 p.m.

If you want, I can also provide a summary of notable stock market or business performance news related to Amazon beyond Prime Day updates. Would you like me to do that?

### Look at the trace

https://platform.openai.com/traces

## 🌟 Setting Up Trader Strategies

- 📝 **Assigning a Strategy:**  
  Each trader is initialized with a specific investment strategy, which is stored in their account.

- 🔄 **Why Store the Strategy?**  
  Storing the strategy allows traders to **change or evolve** their approach over time, giving them flexibility and autonomy.

- 🧑‍💼 **Unique Starting Point:**  
  Every trader begins with a unique strategy to guide their initial decisions.

- 🕹️ **Trader Autonomy:**  
  Traders have the freedom to **adapt or update** their strategy as they see fit, empowering them to respond to market changes and personal insights.

In [9]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-11 00:32:38", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

## 📦 **What Does "Reading a Resource" Mean in MCP?**

When you see us "reading a resource" here, we're actually:

1. 🧑‍💻 **Calling Our MCP Client:**  
- This is the client we built earlier!  
- It acts as a bridge between our code and the MCP server.

2. 🛰️ **MCP Client Talks to the MCP Server:**  
- The client sends a request to the server for a specific resource (like account details or strategy).

3. 🏢 **Server Runs Business Logic:**  
- The MCP server receives the request and executes the business logic to fetch or compute the resource.
- For example, it might look up an account, generate a report, or retrieve a strategy.

4. 📄 **Resource = Useful Text Data:**  
- The server returns the resource as text (e.g., account info, strategy description).
- This text is then available for us to use in our agent's prompt.

---

✨ **Why Use MCP Resources?**

- 🛠️ **Beyond Tools:**  
MCP isn't just for calling tools (functions); it also lets you fetch resources (structured or unstructured data).

- 🧠 **Context for Agents:**  
By reading resources, we can inject up-to-date, relevant information directly into our agent's prompt, giving it the context it needs to make smart decisions.

- 📝 **How Do We Use Them?**  
We simply add the resource text to the system prompt for our agent.  
This way, the agent "knows" about the latest account status, strategy, or any other info we provide.

---

🚀 **In Practice:**  
- Here, we call two MCP resources: one for account details and one for the trader's strategy.
- We then combine these into our system prompt, empowering the agent to act with full context!

In [ ]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name) # Explain how it is calling MCP client -> then it calls the mcp server and it provides the resource by calling businnes logic
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [11]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Ed and your account is under your name, Ed.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-11 00:32:38", 10000.0], ["2025-07-11 00:37:11", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please

### And to run our Trader

In [12]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Summary of Actions Taken:

1. **Market Research**:
   - Investigated current financial news, revealing significant movements in various sectors.
   - **Key Highlights**:
     - **MP Materials Corp. (MP)** jumped over 50% due to a Pentagon investment.
     - Other gainers included **Newegg (NEGG)**, **Hertz (HTZ)**, **Delta Air Lines (DAL)**, and several others in the tech and travel sectors.

2. **Stock Price Checks**:
   - Prices were obtained for the top gainers, leading to investment decisions based on both recent performance and market sentiment.

3. **Trades Executed**:
   - Purchased the following shares:
     - **MP Materials Corp. (MP)**: 100 shares at $30.09.
     - **Newegg Commerce (NEGG)**: 50 shares at $21.76.
     - **Hertz Global Holdings (HTZ)**: 75 shares at $7.14.

### Current Portfolio Overview:
- **Holdings**:
  - **MP**: 100 shares
  - **NEGG**: 50 shares
  - **HTZ**: 75 shares

- **Balance**: $5,367.00
- **Total Portfolio Value**: $9,990.75
- **Total Profit/Loss**: -$9.25 (reflecting current market prices)

### Rationale for Trades:
- **MP** was purchased due to the strategic Pentagon investment.
- **NEGG** was acquired on positive trends in the tech sector.
- **HTZ** was selected based on recovery signals in the travel industry.

Next Steps:
- Continue monitoring these holdings closely for any further news or market actions that may warrant additional trades.

### Then go and look at the trace

http://platform.openai.com/traces


In [13]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "ed", "balance": 5367.0025000000005, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"MP": 100, "NEGG": 50, "HTZ": 75}, "transactions": [{"symbol": "MP", "quantity": 100, "price": 30.09006, "timestamp": "2025-07-11 00:41:20", "rationale": "Strong surge due to Pentagon investment, momentum in rare earth minerals."}, {"symbol": "NEGG", "quantity": 50, "price": 21.76344, "timestamp": "2025-07-11 00:41:20", "rationale": "Notable gains in tech and commerce sector, bullish trend."}, {"symbol": "HTZ", "quantity": 75, "price": 7.14426, "timestamp": "2025-07-11 00:41:20", "rationale": "Strong gain in travel sector, benefiting from post-COVID recovery."}], "portfolio_value_time_series": [["2025-07-11 00:32:38", 10000.0], ["2025-07-11 00:37:11", 10000.0], ["2025-07-11 00:41:20", 9993.993999999999], ["2025-07-11 00:41:20", 9991.822], ["2025-07-11 00:41:20", 9990.7525], ["2025-07-11 00:47:21", 9990.7525]], "tota

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [ ]:
from traders import Trader

In [3]:
trader = Trader("Ed")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("Ed")

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")